In [1]:
# Notes and play code.

In [34]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

query = 'I love tennis'
text = 'andy murry is my favourite tennis player'
max_length = 512

In [35]:
x = tokenizer.encode_plus(text=query,
                          text_pair=text,
                          max_length=max_length,
                          add_special_tokens=True,
                          pad_to_max_length=True)

In [36]:
tokenizer.convert_ids_to_tokens(x['input_ids'])

['<s>',
 'ĠI',
 'Ġlove',
 'Ġtennis',
 '</s>',
 '</s>',
 'Ġand',
 'y',
 'Ġmur',
 'ry',
 'Ġis',
 'Ġmy',
 'Ġfavourite',
 'Ġtennis',
 'Ġplayer',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',


In [21]:
x['input_ids']

[0,
 38,
 657,
 5919,
 2,
 2,
 8,
 219,
 22802,
 1506,
 16,
 127,
 5548,
 5919,
 869,
 2,
 1,
 1]

In [41]:
from transformers import RobertaModel, RobertaConfig

bert  = RobertaModel(RobertaConfig()).from_pretrained('roberta-base')

In [39]:
bert.config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

TypeError: __init__() missing 1 required positional argument: 'config'